# Question 6

Use the TwelveData API to get values of Bitcoin (BTC) and Ethereum (ETH) relative to the US Dollar.
Use closing prices from the last 14 days.

Create a single DataFrame from the results, where each row represents a day, and there are columns for each of Bitcoin and Etherium.
It should look like so:

| date | btc | eth |
| ---- | --- | --- |

Then create a new column, btc_premium, which is the ratio of the Bitcoin price to the Ethereum price (`btc/eth`).
On which day was the Bitcoin premium highest?
On which day the lowest?

## Solution

You "highest" and "lowest" dates will likely be different from mine since you'll have pulled a different fourteen days.

In [1]:
# I'll use getpass again to store my API key
import getpass

api_key = getpass.getpass()

 ································


In [3]:
import requests
URL = 'https://api.twelvedata.com/time_series'

In [14]:
BTC_PARAMS = {
    'symbol': 'ETH/USD',
    'apikey': api_key,
    'interval': '1day',
    'outputsize': '14'
}
ETH_PARAMS = {
    'symbol': 'BTC/USD',
    'apikey': api_key,
    'interval': '1day',
    'outputsize': '14'
}
btc_response = requests.get(URL, params=BTC_PARAMS)
eth_response = requests.get(URL, params=ETH_PARAMS)

In [29]:
btc_values = btc_response.json()['values']
eth_values = eth_response.json()['values']

In [30]:
import pandas as pd

First let's limit each DF to closing prices and name their close columns after the currency they represent.

In [31]:
btc_df = pd.DataFrame(btc_values)
btc_df.head()

,datetime,open,high,low,close
0,2021-06-04,2857.43994,2857.43994,2683.00000,2727.48999
1,2021-06-03,2704.95996,2889.21997,2666.43994,2853.36011
2,2021-06-02,2639.39990,2799.98999,2556.31006,2704.72998
3,2021-06-01,2699.87988,2737.00000,2531.10010,2633.59009
4,2021-05-31,2374.52002,2712.34009,2280.60010,2710.07007


In [32]:
btc_df = btc_df[['datetime', 'close']]

In [33]:
btc_df.columns = ['date', 'btc']

In [34]:
eth_df = pd.DataFrame(eth_values)
eth_df = eth_df[['datetime', 'close']]

In [35]:
eth_df.columns = ['date', 'eth']

Merge the two DataFrames into one:

In [38]:
full_df = pd.merge(btc_df, eth_df, on='date')

In [40]:
full_df.head()

,date,btc,eth
0,2021-06-04,2727.48999,37660.00000
1,2021-06-03,2853.36011,39177.51953
2,2021-06-02,2704.72998,37545.00000
3,2021-06-01,2633.59009,36689.71094
4,2021-05-31,2710.07007,37297.53125


Create our Bitcoin Premium column

In [41]:
full_df['btc_premium'] = full_df['btc'] / full_df['eth']
full_df.head()

TypeError: unsupported operand type(s) for /: 'str' and 'str'

Oh! Our values are strings, not floats.
We'll need to convert them.

In [42]:
full_df['btc'] = full_df['btc'].astype(float)
full_df['eth'] = full_df['eth'].astype(float)

In [43]:
full_df['btc_premium'] = full_df['btc'] / full_df['eth']
full_df.head()

,date,btc,eth,btc_premium
0,2021-06-04,2727.48999,37660.00000,0.072424
1,2021-06-03,2853.36011,39177.51953,0.072832
2,2021-06-02,2704.72998,37545.00000,0.072040
3,2021-06-01,2633.59009,36689.71094,0.071780
4,2021-05-31,2710.07007,37297.53125,0.072661


What day had the highest btc_premium? The lowest?

In [44]:
sorted_df = full_df.sort_values('btc_premium', ascending=True)
sorted_df

,date,btc,eth,btc_premium
12,2021-05-23,2101.88989,34663.05859,0.060638
13,2021-05-22,2289.93994,37418.30078,0.061198
6,2021-05-29,2275.00000,34574.05859,0.065801
5,2021-05-30,2386.92993,35647.76953,0.066959
7,2021-05-28,2418.02002,35654.03125,0.067819
11,2021-05-24,2647.35010,38742.03125,0.068333
10,2021-05-25,2705.19995,38377.42188,0.070489
8,2021-05-27,2743.08008,38381.82031,0.071468
3,2021-06-01,2633.59009,36689.71094,0.071780
2,2021-06-02,2704.72998,37545.00000,0.072040


The lowest, of my 14 days, was May 23. The highest was May 26.